In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/heart-failure-prediction/heart.csv


In [2]:
# Load our dataset
dataset = pd.read_csv('/kaggle/input/heart-failure-prediction/heart.csv')
dataset.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [3]:
# Separate features from targets
features = dataset.drop('HeartDisease',axis=1)
targets = dataset['HeartDisease']

# Encode our targets using one-hot encoding
targets_onehot = pd.get_dummies(targets)
targets_onehot.head()

,0,1
0,1,0
1,0,1
2,1,0
3,0,1
4,1,0


In [4]:
targets.value_counts()

1    508
0    410
Name: HeartDisease, dtype: int64

In [5]:
features_onehot = pd.get_dummies(features)
features_onehot.shape

(918, 20)

In [6]:
features_onehot = features_onehot.astype('float32')
targets_onehot = targets_onehot.astype('float32')
features_onehot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Age                918 non-null    float32
 1   RestingBP          918 non-null    float32
 2   Cholesterol        918 non-null    float32
 3   FastingBS          918 non-null    float32
 4   MaxHR              918 non-null    float32
 5   Oldpeak            918 non-null    float32
 6   Sex_F              918 non-null    float32
 7   Sex_M              918 non-null    float32
 8   ChestPainType_ASY  918 non-null    float32
 9   ChestPainType_ATA  918 non-null    float32
 10  ChestPainType_NAP  918 non-null    float32
 11  ChestPainType_TA   918 non-null    float32
 12  RestingECG_LVH     918 non-null    float32
 13  RestingECG_Normal  918 non-null    float32
 14  RestingECG_ST      918 non-null    float32
 15  ExerciseAngina_N   918 non-null    float32
 16  ExerciseAngina_Y   918 non

In [7]:
from sklearn.model_selection import train_test_split

train_features, test_features, train_targets, test_targets = train_test_split(features_onehot, targets_onehot, test_size=0.1, stratify=targets)

In [8]:
import tensorflow as tf

train_features_tensor = tf.convert_to_tensor(train_features)
train_targets_tensor = tf.convert_to_tensor(train_targets)
test_features_tensor = tf.convert_to_tensor(test_features)
test_targets_tensor = tf.convert_to_tensor(test_targets)

test_targets.shape

(92, 2)

In [9]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([Dense(20,activation='sigmoid'), Dense(12,activation='sigmoid'), Dense(2,activation="sigmoid")])
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [10]:
model.fit(train_features, train_targets, epochs=10, batch_size=24)

Epoch 1/10
35/35 [==============================] - 1s 2ms/step - loss: 0.6718 - accuracy: 0.5533
Epoch 2/10
35/35 [==============================] - 0s 2ms/step - loss: 0.6571 - accuracy: 0.5847
Epoch 3/10
35/35 [==============================] - 0s 2ms/step - loss: 0.6479 - accuracy: 0.5932
Epoch 4/10
35/35 [==============================] - 0s 2ms/step - loss: 0.6411 - accuracy: 0.6501
Epoch 5/10
35/35 [==============================] - 0s 2ms/step - loss: 0.6351 - accuracy: 0.6743
Epoch 6/10
35/35 [==============================] - 0s 2ms/step - loss: 0.6302 - accuracy: 0.6864
Epoch 7/10
35/35 [==============================] - 0s 2ms/step - loss: 0.6256 - accuracy: 0.6985
Epoch 8/10
35/35 [==============================] - 0s 2ms/step - loss: 0.6208 - accuracy: 0.6961
Epoch 9/10
35/35 [==============================] - 0s 2ms/step - loss: 0.6196 - accuracy: 0.7022
Epoch 10/10
35/35 [==============================] - 0s 2ms/step - loss: 0.6122 - accuracy: 0.7288


In [11]:
model.evaluate(test_features, test_targets)

3/3 [==============================] - 0s 3ms/step - loss: 0.5507 - accuracy: 0.7935


[0.5507282018661499, 0.79347825050354]